In [ ]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame

def visualize(im, detections, poses3d, poses2d, edges):
    """Visualize 2D and 3D poses with bounding boxes."""
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(im)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)

    # Collect all coordinates to determine dynamic axis limits
    all_x_3d, all_y_3d, all_z_3d = [], [], []
    for pose3d in poses3d:
        all_x_3d.extend(pose3d[:, 0])
        all_y_3d.extend(pose3d[:, 1])
        all_z_3d.extend(pose3d[:, 2])

    # Set dynamic axis limits with padding
    if all_x_3d:
        x_min, x_max = min(all_x_3d), max(all_x_3d)
        y_min, y_max = min(all_y_3d), max(all_y_3d)
        z_min, z_max = min(all_z_3d), max(all_z_3d)
        padding = 0.5  # Add padding in meters
        pose_ax.set_xlim(x_min - padding, x_max + padding)
        pose_ax.set_ylim(y_min - padding, y_max + padding)
        pose_ax.set_zlim(z_min - padding, z_max + padding)
    else:
        pose_ax.set_xlim(-1.5, 1.5)
        pose_ax.set_ylim(-1.5, 1.5)
        pose_ax.set_zlim(0, 3.0)

    pose_ax.set_xlabel('X (m)')
    pose_ax.set_ylabel('Y (m)')
    pose_ax.set_zlabel('Z (m)')

    # Swap Y and Z axes and negate Z for Metrabs coordinate system
    for pose3d, pose2d in zip(poses3d, poses2d):
        pose3d_adjusted = pose3d.copy()
        pose3d_adjusted[:, 1], pose3d_adjusted[:, 2] = pose3d[:, 2], -pose3d[:, 1]
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d_adjusted[i_start], pose3d_adjusted[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d_adjusted.T, s=2)

    fig.tight_layout()
    plt.show()


#def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", frame_interval=10, output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, visualize 3D poses, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_2d = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    # Initialize 3D video output
    output_3d_video = "output_3d_video.mp4"
    out_3d = cv2.VideoWriter(output_3d_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    # Create temporary directory for 3D plot images
    temp_dir = "temp_3d_frames"
    os.makedirs(temp_dir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()
                boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()  # Get bounding boxes if available

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Visualize 2D and 3D poses (optional, for debugging)
                    # visualize(frame_rgb_resized, boxes, keypoints_3d, keypoints_2d[:, :, :2], skeleton_edges)

                    # Assign person IDs (simple assignment based on detection order)
                    person_ids = list(range(keypoints_2d.shape[0]))

                    # Render 3D skeleton and write to 3D video
                    if keypoints_3d.size > 0:
                        temp_image_path = render_3d_skeleton(keypoints_3d, person_ids, temp_dir, frame_count, frame_width, frame_height, skeleton_edges)
                        if temp_image_path:
                            plot_image = cv2.imread(temp_image_path)
                            if plot_image is not None:
                                plot_image = cv2.resize(plot_image, (frame_width, frame_height))
                                out_3d.write(plot_image)
                                print(f"Successfully wrote frame {frame_count} to 3D video")
                            else:
                                print(f"Failed to read 3D plot image for frame {frame_count}")
                    else:
                        blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                        out_3d.write(blank_frame)
                        print(f"Wrote blank frame {frame_count} to 3D video (no 3D landmarks)")

                    # Iterate over detected poses (persons)
                    for i in range(keypoints_2d.shape[0]):
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            # Draw skeleton
                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Add person label (e.g., "Person 1", "Person 2", etc.)
                            person_label = f"Person {i + 1}"  # Person ID starts from 1
                            label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head
                            cv2.putText(frame, person_label, label_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out_2d.write(frame)  # Write frame to 2D video
        frame_count += 1

    cap.release()
    out_2d.release()
    out_3d.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

    # Clean up temporary 3D plot images
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temporary directory {temp_dir}")

    return df
def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", frame_interval=10, output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, visualize 3D poses, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_2d = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    # Initialize 3D video output
    output_3d_video = "output_3d_video.mp4"
    out_3d = cv2.VideoWriter(output_3d_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    # Create temporary directory for 3D plot images
    temp_dir = "temp_3d_frames"
    os.makedirs(temp_dir, exist_ok=True)

    # Pre-generate 3D frames and store them
    temp_image_paths = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()
                boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()  # Get bounding boxes if available

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Assign person IDs (simple assignment based on detection order)
                    person_ids = list(range(keypoints_2d.shape[0]))

                    # Render 3D skeleton and store the image path
                    if keypoints_3d.size > 0:
                        temp_image_path = render_3d_skeleton(keypoints_3d, person_ids, temp_dir, frame_count, frame_width, frame_height, skeleton_edges)
                        if temp_image_path:
                            temp_image_paths.append(temp_image_path)
                            print(f"Generated 3D frame for frame {frame_count}")
                    else:
                        blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_count}.png")
                        cv2.imwrite(temp_image_path, blank_frame)
                        temp_image_paths.append(temp_image_path)
                        print(f"Generated blank 3D frame for frame {frame_count}")

                    # Iterate over detected poses (persons)
                    for i in range(keypoints_2d.shape[0]):
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            # Draw skeleton
                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Add person label (e.g., "Person 1", "Person 2", etc.)
                            person_label = f"Person {i + 1}"  # Person ID starts from 1
                            label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head
                            cv2.putText(frame, person_label, label_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out_2d.write(frame)  # Write frame to 2D video
        frame_count += 1

    cap.release()
    out_2d.release()

    # Write 3D video with frame duplication to match original duration
    for i in range(total_frames):
        frame_idx = i // (frame_interval if frame_interval > 0 else 1)  # Map to nearest rendered frame
        if frame_idx < len(temp_image_paths):
            plot_image = cv2.imread(temp_image_paths[frame_idx])
            if plot_image is not None:
                plot_image = cv2.resize(plot_image, (frame_width, frame_height))
                out_3d.write(plot_image)
                print(f"Wrote frame {i} to 3D video (from {temp_image_paths[frame_idx]})")
            else:
                blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                out_3d.write(blank_frame)
                print(f"Wrote blank frame {i} to 3D video (image load failed)")
        else:
            blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
            out_3d.write(blank_frame)
            print(f"Wrote blank frame {i} to 3D video (no corresponding 3D frame)")

    out_3d.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

    # Clean up temporary 3D plot images
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temporary directory {temp_dir}")

    return df
#def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, visualize 3D poses, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_2d = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    # Initialize 3D video output
    output_3d_video = "output_3d_video.mp4"
    out_3d = cv2.VideoWriter(output_3d_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    # Create temporary directory for 3D plot images
    temp_dir = "temp_3d_frames"
    os.makedirs(temp_dir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_height, original_width = frame.shape[:2]
        frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
        image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

        # Run pose estimation for every frame
        pred = model.detect_poses(image_tensor, skeleton='smpl_24')

        if 'poses2d' in pred and 'poses3d' in pred:
            keypoints_2d = pred['poses2d'].numpy()
            keypoints_3d = pred['poses3d'].numpy()
            boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()  # Get bounding boxes if available

            if keypoints_2d.shape[0] == 0:
                print(f"No pose detected in frame {frame_count}")
            else:
                # Assign person IDs (simple assignment based on detection order)
                person_ids = list(range(keypoints_2d.shape[0]))

                # Render 3D skeleton and write to 3D video
                if keypoints_3d.size > 0:
                    temp_image_path = render_3d_skeleton(keypoints_3d, person_ids, temp_dir, frame_count, frame_width, frame_height, skeleton_edges)
                    if temp_image_path:
                        plot_image = cv2.imread(temp_image_path)
                        if plot_image is not None:
                            plot_image = cv2.resize(plot_image, (frame_width, frame_height))
                            out_3d.write(plot_image)
                            print(f"Successfully wrote frame {frame_count} to 3D video")
                        else:
                            print(f"Failed to read 3D plot image for frame {frame_count}")
                    else:
                        blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                        out_3d.write(blank_frame)
                        print(f"Wrote blank frame {frame_count} to 3D video (render failed)")
                else:
                    blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                    out_3d.write(blank_frame)
                    print(f"Wrote blank frame {frame_count} to 3D video (no 3D landmarks)")

                # Iterate over detected poses (persons)
                for i in range(keypoints_2d.shape[0]):
                    if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                        kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                        kpt_3d = keypoints_3d[i]

                        # Scale keypoints to original frame size
                        kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                        kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                        # Draw skeleton
                        frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                        # Add person label (e.g., "Person 1", "Person 2", etc.)
                        person_label = f"Person {i + 1}"  # Person ID starts from 1
                        label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head
                        cv2.putText(frame, person_label, label_position, 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                        # Store frame-wise keypoints
                        for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                            data_list.append([
                                frame_count, 
                                i, 
                                j, 
                                kpt_2d[j][0], 
                                kpt_2d[j][1], 
                                kpt_3d[j][0], 
                                kpt_3d[j][1], 
                                kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                            ])
                    else:
                        print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out_2d.write(frame)  # Write frame to 2D video
        frame_count += 1

    cap.release()
    out_2d.release()
    out_3d.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

    # Clean up temporary 3D plot images
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temporary directory {temp_dir}")

    return df
#def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, visualize 3D poses, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_2d = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    # Initialize 3D video output
    output_3d_video = "output_3d_video.mp4"
    out_3d = cv2.VideoWriter(output_3d_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    # Create temporary directory for 3D plot images
    temp_dir = "temp_3d_frames"
    os.makedirs(temp_dir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_height, original_width = frame.shape[:2]
        frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
        image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

        # Run pose estimation for every frame
        pred = model.detect_poses(image_tensor, skeleton='smpl_24')

        if 'poses2d' in pred and 'poses3d' in pred:
            keypoints_2d = pred['poses2d'].numpy()
            keypoints_3d = pred['poses3d'].numpy()
            boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()  # Get bounding boxes if available

            if keypoints_2d.shape[0] == 0:
                print(f"No pose detected in frame {frame_count}")
            else:
                # Assign person IDs (simple assignment based on detection order)
                person_ids = list(range(keypoints_2d.shape[0]))

                # Render 3D skeleton and write to 3D video
                if keypoints_3d.size > 0:
                    temp_image_path = render_3d_skeleton(keypoints_3d, person_ids, temp_dir, frame_count, frame_width, frame_height, skeleton_edges)
                    if temp_image_path:
                        plot_image = cv2.imread(temp_image_path)
                        if plot_image is not None:
                            # Remove resizing to preserve original plot dimensions
                            out_3d.write(plot_image)
                            print(f"Successfully wrote frame {frame_count} to 3D video")
                        else:
                            print(f"Failed to read 3D plot image for frame {frame_count}")
                    else:
                        blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                        out_3d.write(blank_frame)
                        print(f"Wrote blank frame {frame_count} to 3D video (render failed)")
                else:
                    blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                    out_3d.write(blank_frame)
                    print(f"Wrote blank frame {frame_count} to 3D video (no 3D landmarks)")

                # Iterate over detected poses (persons)
                for i in range(keypoints_2d.shape[0]):
                    if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                        kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                        kpt_3d = keypoints_3d[i]

                        # Scale keypoints to original frame size
                        kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                        kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                        # Draw skeleton
                        frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                        # Add person label (e.g., "Person 1", "Person 2", etc.)
                        person_label = f"Person {i + 1}"  # Person ID starts from 1
                        label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head
                        cv2.putText(frame, person_label, label_position, 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                        # Store frame-wise keypoints
                        for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                            data_list.append([
                                frame_count, 
                                i, 
                                j, 
                                kpt_2d[j][0], 
                                kpt_2d[j][1], 
                                kpt_3d[j][0], 
                                kpt_3d[j][1], 
                                kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                            ])
                    else:
                        print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out_2d.write(frame)  # Write frame to 2D video
        frame_count += 1

    cap.release()
    out_2d.release()
    out_3d.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

    # Clean up temporary 3D plot images
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"Cleaned up temporary directory {temp_dir}")

    return df
def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (mm)': stride_length_left_2d,
        'Stride Length Right (mm)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (mm/s)': gait_speed_left_2d,
        'Gait Speed Right (mm/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }
def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (mm)': stride_length_left_3d,
        'Stride Length Right (mm)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (mm/s)': gait_speed_left_3d,
        'Gait Speed Right (mm/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }
#def render_3d_skeleton(poses3d, person_ids, temp_dir, frame_idx, frame_width, frame_height, skeleton_edges):
    """Render the 3D skeleton as an image for a single frame."""
    try:
        fig = plt.figure(figsize=(frame_width / 100, frame_height / 100), dpi=100)  # Match video dimensions
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(5, -85)

        # Collect all coordinates to determine dynamic axis limits
        all_x_3d, all_y_3d, all_z_3d = [], [], []
        for pose3d in poses3d:
            all_x_3d.extend(pose3d[:, 0])
            all_y_3d.extend(pose3d[:, 1])
            all_z_3d.extend(pose3d[:, 2])

        # Set dynamic axis limits with padding
        if all_x_3d:
            x_min, x_max = min(all_x_3d), max(all_x_3d)
            y_min, y_max = min(all_y_3d), max(all_y_3d)
            z_min, z_max = min(all_z_3d), max(all_z_3d)
            padding = 0.5  # Add padding in meters (adjust based on Metrabs scale)
            ax.set_xlim(x_min - padding, x_max + padding)
            ax.set_ylim(y_min - padding, y_max + padding)
            ax.set_zlim(z_min - padding, z_max + padding)
        else:
            ax.set_xlim(-1.5, 1.5)
            ax.set_ylim(-1.5, 1.5)
            ax.set_zlim(0, 3.0)

        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Plot 3D skeleton for each person
        for pose3d, person_id in zip(poses3d, person_ids):
            if person_id == -1:
                continue
            # Plot skeleton connections
            for i_start, i_end in skeleton_edges:
                ax.plot(*zip(pose3d[i_start], pose3d[i_end]), 'g-', marker='o', markersize=2)
            ax.scatter(*pose3d.T, c='g', s=2)
            # Add person label above the head (first joint, e.g., pelvis)
            ax.text(pose3d[0, 0], pose3d[0, 1], pose3d[0, 2] + 0.2, 
                    f"Person {person_id + 1}", color='red')

        fig.tight_layout()
        # Save the plot as an image
        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_idx}.png")
        plt.savefig(temp_image_path, bbox_inches='tight')
        plt.close()

        return temp_image_path
    except Exception as e:
        print(f"Error rendering 3D skeleton for frame {frame_idx}: {str(e)}")
        return None
def render_3d_skeleton(poses3d, person_ids, temp_dir, frame_idx, frame_width, frame_height, skeleton_edges):
    """Render the 3D skeleton as an image for a single frame."""
    try:
        fig = plt.figure(figsize=(frame_width / 100, frame_height / 100), dpi=100)  # Match video dimensions
        ax = fig.add_subplot(111, projection='3d')

        # Adjust view angle for a more natural perspective (front-side view)
        ax.view_init(elev=20, azim=45)  # elev=20 for slight top-down, azim=45 for front-side view

        # Collect all coordinates to determine dynamic axis limits
        all_x_3d, all_y_3d, all_z_3d = [], [], []
        for pose3d in poses3d:
            # Apply axis transformation: swap Y and Z, negate Z to match standard orientation
            pose3d_adjusted = pose3d.copy()
            pose3d_adjusted[:, 1], pose3d_adjusted[:, 2] = pose3d[:, 2], -pose3d[:, 1]
            all_x_3d.extend(pose3d_adjusted[:, 0])
            all_y_3d.extend(pose3d_adjusted[:, 1])
            all_z_3d.extend(pose3d_adjusted[:, 2])

        # Set dynamic axis limits with padding
        if all_x_3d:
            x_min, x_max = min(all_x_3d), max(all_x_3d)
            y_min, y_max = min(all_y_3d), max(all_y_3d)
            z_min, z_max = min(all_z_3d), max(all_z_3d)
            padding = 0.5  # Add padding in meters
            ax.set_xlim(x_min - padding, x_max + padding)
            ax.set_ylim(y_min - padding, y_max + padding)
            ax.set_zlim(z_min - padding, z_max + padding)
        else:
            ax.set_xlim(-1.5, 1.5)
            ax.set_ylim(-1.5, 1.5)
            ax.set_zlim(0, 3.0)

        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Plot 3D skeleton for each person
        for pose3d, person_id in zip(poses3d, person_ids):
            if person_id == -1:
                continue
            # Apply axis transformation for plotting
            pose3d_adjusted = pose3d.copy()
            pose3d_adjusted[:, 1], pose3d_adjusted[:, 2] = pose3d[:, 2], -pose3d[:, 1]
            # Plot skeleton connections
            for i_start, i_end in skeleton_edges:
                ax.plot(*zip(pose3d_adjusted[i_start], pose3d_adjusted[i_end]), 'g-', marker='o', markersize=2)
            ax.scatter(*pose3d_adjusted.T, c='g', s=2)
            # Add person label above the head (first joint, e.g., pelvis)
            ax.text(pose3d_adjusted[0, 0], pose3d_adjusted[0, 1], pose3d_adjusted[0, 2] + 0.2, 
                    f"Person {person_id + 1}", color='red')

        fig.tight_layout()
        # Save the plot as an image
        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_idx}.png")
        plt.savefig(temp_image_path, bbox_inches='tight')
        plt.close()

        return temp_image_path
    except Exception as e:
        print(f"Error rendering 3D skeleton for frame {frame_idx}: {str(e)}")
        return None  
#def render_3d_skeleton(poses3d, person_ids, temp_dir, frame_idx, frame_width, frame_height, skeleton_edges):
    """Render the 3D skeleton as an image for a single frame."""
    try:
        # Use a fixed figsize to ensure consistent 3D aspect ratio (adjust as needed)
        fig = plt.figure(figsize=(10, 10), dpi=100)  # Square figure for better 3D proportion
        ax = fig.add_subplot(111, projection='3d')

        # Set view angle for a front-facing perspective
        ax.view_init(elev=15, azim=180)  # elev=15 for slight top-down, azim=180 for front view

        # Collect all coordinates to determine dynamic axis limits
        all_x_3d, all_y_3d, all_z_3d = [], [], []
        for pose3d in poses3d:
            # Adjust axis transformation: swap Y and Z, negate Z for height, and ensure front faces viewer
            pose3d_adjusted = pose3d.copy()
            # New transformation: X (width), Y (height), Z (depth), with front facing positive Z
            pose3d_adjusted[:, [1, 2]] = pose3d[:, [2, 1]]  # Swap Y and Z
            pose3d_adjusted[:, 2] = -pose3d_adjusted[:, 2]  # Negate Z for depth (front faces viewer)
            all_x_3d.extend(pose3d_adjusted[:, 0])
            all_y_3d.extend(pose3d_adjusted[:, 1])
            all_z_3d.extend(pose3d_adjusted[:, 2])

        # Set dynamic axis limits with padding and equal aspect ratio
        if all_x_3d:
            x_min, x_max = min(all_x_3d), max(all_x_3d)
            y_min, y_max = min(all_y_3d), max(all_y_3d)
            z_min, z_max = min(all_z_3d), max(all_z_3d)
            padding = 0.5  # Add padding in meters
            ax.set_xlim(x_min - padding, x_max + padding)
            ax.set_ylim(y_min - padding, y_max + padding)
            ax.set_zlim(z_min - padding, z_max + padding)
            # Ensure equal aspect ratio for undistorted 3D view
            ax.set_box_aspect([1, 1, 1])  # Equal scaling on all axes
        else:
            ax.set_xlim(-1.5, 1.5)
            ax.set_ylim(-1.5, 1.5)
            ax.set_zlim(0, 3.0)
            ax.set_box_aspect([1, 1, 1])  # Default equal aspect ratio

        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Plot 3D skeleton for each person
        for pose3d, person_id in zip(poses3d, person_ids):
            if person_id == -1:
                continue
            # Apply axis transformation for plotting
            pose3d_adjusted = pose3d.copy()
            pose3d_adjusted[:, [1, 2]] = pose3d[:, [2, 1]]  # Swap Y and Z
            pose3d_adjusted[:, 2] = -pose3d_adjusted[:, 2]  # Negate Z for depth
            # Plot skeleton connections
            for i_start, i_end in skeleton_edges:
                ax.plot(*zip(pose3d_adjusted[i_start], pose3d_adjusted[i_end]), 'g-', marker='o', markersize=2)
            ax.scatter(*pose3d_adjusted.T, c='g', s=2)
            # Add person label above the head (first joint, e.g., pelvis)
            ax.text(pose3d_adjusted[0, 0], pose3d_adjusted[0, 1], pose3d_adjusted[0, 2] + 0.2, 
                    f"Person {person_id + 1}", color='red')

        fig.tight_layout()
        # Save the plot as an image
        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_idx}.png")
        plt.savefig(temp_image_path, bbox_inches='tight')
        plt.close()

        return temp_image_path
    except Exception as e:
        print(f"Error rendering 3D skeleton for frame {frame_idx}: {str(e)}")
        return None
#def render_3d_skeleton(poses3d, person_ids, temp_dir, frame_idx, frame_width, frame_height, skeleton_edges):
    """Render the 3D skeleton as an image for a single frame."""
    try:
        # Calculate aspect ratio and set figure size based on video dimensions
        aspect_ratio = frame_width / frame_height
        fig_width = 10  # Base width in inches
        fig_height = fig_width / aspect_ratio  # Adjust height to match aspect ratio
        fig = plt.figure(figsize=(fig_width, fig_height), dpi=150)  # Higher DPI for better quality
        ax = fig.add_subplot(111, projection='3d')

        # Set view angle for a front-facing perspective
        ax.view_init(elev=15, azim=180)  # Front view, as corrected previously

        # Collect all coordinates to determine dynamic axis limits
        all_x_3d, all_y_3d, all_z_3d = [], [], []
        for pose3d in poses3d:
            # Adjust axis transformation: swap Y and Z, negate Z for height, ensure front faces viewer
            pose3d_adjusted = pose3d.copy()
            pose3d_adjusted[:, [1, 2]] = pose3d[:, [2, 1]]  # Swap Y and Z
            pose3d_adjusted[:, 2] = -pose3d_adjusted[:, 2]  # Negate Z for depth
            all_x_3d.extend(pose3d_adjusted[:, 0])
            all_y_3d.extend(pose3d_adjusted[:, 1])
            all_z_3d.extend(pose3d_adjusted[:, 2])

        # Set dynamic axis limits with padding and equal aspect ratio
        if all_x_3d:
            x_min, x_max = min(all_x_3d), max(all_x_3d)
            y_min, y_max = min(all_y_3d), max(all_y_3d)
            z_min, z_max = min(all_z_3d), max(all_z_3d)
            padding = 0.5  # Add padding in meters
            ax.set_xlim(x_min - padding, x_max + padding)
            ax.set_ylim(y_min - padding, y_max + padding)
            ax.set_zlim(z_min - padding, z_max + padding)
            ax.set_box_aspect([1, 1, 1])  # Equal scaling on all axes
        else:
            ax.set_xlim(-1.5, 1.5)
            ax.set_ylim(-1.5, 1.5)
            ax.set_zlim(0, 3.0)
            ax.set_box_aspect([1, 1, 1])  # Default equal aspect ratio

        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Plot 3D skeleton for each person
        for pose3d, person_id in zip(poses3d, person_ids):
            if person_id == -1:
                continue
            # Apply axis transformation for plotting
            pose3d_adjusted = pose3d.copy()
            pose3d_adjusted[:, [1, 2]] = pose3d[:, [2, 1]]  # Swap Y and Z
            pose3d_adjusted[:, 2] = -pose3d_adjusted[:, 2]  # Negate Z for depth
            # Plot skeleton connections
            for i_start, i_end in skeleton_edges:
                ax.plot(*zip(pose3d_adjusted[i_start], pose3d_adjusted[i_end]), 'g-', marker='o', markersize=2)
            ax.scatter(*pose3d_adjusted.T, c='g', s=2)
            # Add person label above the head (first joint, e.g., pelvis)
            ax.text(pose3d_adjusted[0, 0], pose3d_adjusted[0, 1], pose3d_adjusted[0, 2] + 0.2, 
                    f"Person {person_id + 1}", color='red')

        fig.tight_layout()
        # Save the plot at the target resolution without resizing
        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_idx}.png")
        plt.savefig(temp_image_path, dpi=150, bbox_inches='tight')  # Match video DPI
        plt.close()

        return temp_image_path
    except Exception as e:
        print(f"Error rendering 3D skeleton for frame {frame_idx}: {str(e)}")
        return None
def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\Mediapipe\\walk.mp4"
    output_excel = "2dand3dposedatawithlabelschanged.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video (this generates the initial Excel file with 2D in pixels, 3D in mm)
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)

    # Step 1: Compute pixel-to-mm conversion factor using hip-to-ankle distance
    unique_persons = df['Person'].unique()
    pixel_to_mm_factors = []

    for person_id in unique_persons:
        df_person = df[df['Person'] == person_id]
        
        # Extract hip and ankle coordinates for all frames
        left_hip_2d = df_person[df_person['Joint'] == 1][['Frame', 'X_2D', 'Y_2D']]
        left_ankle_2d = df_person[df_person['Joint'] == 7][['Frame', 'X_2D', 'Y_2D']]
        left_hip_3d = df_person[df_person['Joint'] == 1][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]
        left_ankle_3d = df_person[df_person['Joint'] == 7][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]

        # Merge on Frame to get corresponding hip and ankle coordinates
        hip_ankle_2d = left_hip_2d.merge(left_ankle_2d, on='Frame', suffixes=('_hip', '_ankle'))
        hip_ankle_3d = left_hip_3d.merge(left_ankle_3d, on='Frame', suffixes=('_hip', '_ankle'))

        # Compute distances for each frame
        distances_2d = []
        distances_3d = []
        for idx in range(len(hip_ankle_2d)):
            # 2D distance (pixels)
            hip_2d = hip_ankle_2d.iloc[idx][['X_2D_hip', 'Y_2D_hip']].values
            ankle_2d = hip_ankle_2d.iloc[idx][['X_2D_ankle', 'Y_2D_ankle']].values
            dist_2d = distance.euclidean(hip_2d, ankle_2d)

            # 3D distance (mm)
            hip_3d = hip_ankle_3d.iloc[idx][['X_3D_hip', 'Y_3D_hip', 'Z_3D_hip']].values
            ankle_3d = hip_ankle_3d.iloc[idx][['X_3D_ankle', 'Y_3D_ankle', 'Z_3D_ankle']].values
            dist_3d = distance.euclidean(hip_3d, ankle_3d)

            if dist_2d > 0:  # Avoid division by zero
                pixel_to_mm = dist_3d / dist_2d
                pixel_to_mm_factors.append(pixel_to_mm)
                distances_2d.append(dist_2d)
                distances_3d.append(dist_3d)

    # Compute the average pixel-to-mm factor
    if pixel_to_mm_factors:
        pixel_to_mm = np.mean(pixel_to_mm_factors)
        print(f"Computed pixel-to-mm conversion factor: {pixel_to_mm:.4f} mm/pixel")
        print(f"Average 3D hip-to-ankle distance: {np.mean(distances_3d):.2f} mm")
        print(f"Average 2D hip-to-ankle distance: {np.mean(distances_2d):.2f} pixels")
    else:
        # Fallback to anthropometric average if no valid distances are computed
        pixel_to_mm = 900 / 500  # Assume 900 mm leg length, 500 pixels in 2D (rough estimate)
        print(f"No valid distances computed. Using fallback pixel-to-mm factor: {pixel_to_mm:.4f} mm/pixel")

    # Step 2: Convert 2D coordinates to millimeters
    df['X_2D'] = df['X_2D'] * pixel_to_mm
    df['Y_2D'] = df['Y_2D'] * pixel_to_mm

    # Step 3: Save the updated Excel file with 2D and 3D coordinates in millimeters
    updated_excel = "2dand3dposedatawithlabelschanged_converted.xlsx"
    df.to_excel(updated_excel, index=False)
    print(f"Updated pose data with 2D coordinates in millimeters saved as {updated_excel}")

    # Step 4: Calculate gait metrics
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    print(f"Detected {len(unique_persons)} persons in the video.")

    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics (now in millimeters)
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics (already in millimeters)
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all detected persons saved to 'gait_metrics_per_person.xlsx'")
if __name__ == '__main__':
    main()

Model loaded successfully!
Generated 3D frame for frame 0
Generated 3D frame for frame 10
Generated 3D frame for frame 20
Generated 3D frame for frame 30
Generated 3D frame for frame 40
Generated 3D frame for frame 50
Generated 3D frame for frame 60
Generated 3D frame for frame 70
Generated 3D frame for frame 80
Generated 3D frame for frame 90
Generated 3D frame for frame 100
Generated 3D frame for frame 110
Generated 3D frame for frame 120
No pose detected in frame 130
Generated 3D frame for frame 140
Wrote frame 0 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 1 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 2 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 3 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 4 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 5 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 6 to 3D video (from temp_3d_frames\3d_frame_0.png)
Wrote frame 7 to 3D video (from temp_3d_frames\3d_f

In [18]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import shutil

def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)
        else:
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)
    return frame

def visualize(im, detections, poses3d, poses2d, edges, frame_idx, plot_dir, display=False):
    """Visualize 2D and 3D poses with bounding boxes and save to plot_dir."""
    os.makedirs(plot_dir, exist_ok=True)
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(im)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False, edgecolor='r'))
    
    for pose2d in poses2d:
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), 'b-', marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, c='r', s=2)
    
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    
    all_x_3d, all_y_3d, all_z_3d = [], [], []
    for pose3d in poses3d:
        x_3d = pose3d[:, 0] * 1000  # Convert meters to mm
        y_3d = pose3d[:, 2] * 1000  # Swap y and z, convert to mm
        z_3d = -pose3d[:, 1] * 1000  # Negate and convert to mm
        all_x_3d.extend(x_3d)
        all_y_3d.extend(y_3d)
        all_z_3d.extend(z_3d)
    
    if all_x_3d:
        padding = 500
        pose_ax.set_xlim(min(all_x_3d) - padding, max(all_x_3d) + padding)
        pose_ax.set_ylim(min(all_y_3d) - padding, max(all_y_3d) + padding)
        pose_ax.set_zlim(min(all_z_3d) - padding, max(all_z_3d) + padding)
    else:
        pose_ax.set_xlim(-1500, 1500)
        pose_ax.set_ylim(-1500, 1500)
        pose_ax.set_zlim(0, 3000)
    
    pose_ax.set_xlabel('X (mm)')
    pose_ax.set_ylabel('Y (mm)')
    pose_ax.set_zlabel('Z (mm)')
    
    for pose3d in poses3d:
        x_3d = pose3d[:, 0] * 1000
        y_3d = pose3d[:, 2] * 1000
        z_3d = -pose3d[:, 1] * 1000
        for i_start, i_end in edges:
            pose_ax.plot([x_3d[i_start], x_3d[i_end]], [y_3d[i_start], y_3d[i_end]], 
                         [z_3d[i_start], z_3d[i_end]], 'g-', marker='o', markersize=2)
        pose_ax.scatter(x_3d, y_3d, z_3d, c='g', s=2)
    
    fig.tight_layout()
    plot_path = os.path.join(plot_dir, f"side_by_side_frame_{frame_idx}.png")
    plt.savefig(plot_path)
    if display:
        plt.show()
    plt.close()

def render_3d_skeleton(poses3d, person_ids, edges, temp_dir, frame_idx, frame_width, frame_height):
    """Render 3D skeleton as an image for a single frame."""
    try:
        os.makedirs(temp_dir, exist_ok=True)
        fig = plt.figure(figsize=(frame_width / 100, frame_height / 100), dpi=100)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(5, -85)
        
        all_x_3d, all_y_3d, all_z_3d = [], [], []
        for pose3d in poses3d:
            x_3d = pose3d[:, 0] * 1000
            y_3d = pose3d[:, 2] * 1000
            z_3d = -pose3d[:, 1] * 1000
            all_x_3d.extend(x_3d)
            all_y_3d.extend(y_3d)
            all_z_3d.extend(z_3d)
        
        if all_x_3d:
            padding = 500
            ax.set_xlim(min(all_x_3d) - padding, max(all_x_3d) + padding)
            ax.set_ylim(min(all_y_3d) - padding, max(all_y_3d) + padding)
            ax.set_zlim(min(all_z_3d) - padding, max(all_z_3d) + padding)
        else:
            ax.set_xlim(-1500, 1500)
            ax.set_ylim(-1500, 1500)
            ax.set_zlim(0, 3000)
        
        ax.set_xlabel('X (mm)')
        ax.set_ylabel('Y (mm)')
        ax.set_zlabel('Z (mm)')
        
        for pose3d, person_id in zip(poses3d, person_ids):
            if person_id == -1:
                continue
            x_3d = pose3d[:, 0] * 1000
            y_3d = pose3d[:, 2] * 1000
            z_3d = -pose3d[:, 1] * 1000
            for i_start, i_end in edges:
                ax.plot([x_3d[i_start], x_3d[i_end]], [y_3d[i_start], y_3d[i_end]], 
                        [z_3d[i_start], z_3d[i_end]], 'g-', marker='o', markersize=2)
            ax.scatter(x_3d, y_3d, z_3d, c='g', s=2)
            # Add person label above head (joint 0: pelvis)
            ax.text(x_3d[0], y_3d[0], z_3d[0] + 200, f"Person {person_id + 1}", color='red')
        
        fig.tight_layout()
        temp_image_path = os.path.join(temp_dir, f"3d_frame_{frame_idx}.png")
        plt.savefig(temp_image_path, bbox_inches='tight')
        plt.close()
        return temp_image_path
    except Exception as e:
        print(f"Error rendering 3D skeleton for frame {frame_idx}: {str(e)}")
        return None

def process_video(video_path, model, skeleton_edges, output_2d_video="output_video3d_withlabelschanged.mp4", 
                 output_3d_video="output_3d_pose.mp4", frame_interval=10, output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, generate 2D and 3D videos, and save coordinates."""
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps <= 0:
        fps = 30
        print("Invalid FPS. Using default FPS of 30.")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_2d = cv2.VideoWriter(output_2d_video, fourcc, fps, (frame_width, frame_height))
    out_3d = cv2.VideoWriter(output_3d_video, fourcc, fps, (frame_width, frame_height))
    if not out_2d.isOpened() or not out_3d.isOpened():
        raise ValueError("Could not initialize VideoWriter. Check codec or permissions.")

    temp_dir = "temp_3d_frames"
    os.makedirs(temp_dir, exist_ok=True)
    
    frame_count = 0
    data_list = []
    plot_dir = "3d_plots"

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_height, original_width = frame.shape[:2]
        frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
        image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

        pred = model.detect_poses(image_tensor, skeleton='smpl_24')
        person_ids = list(range(pred['poses2d'].shape[0]))  # Simple person ID assignment

        if 'poses2d' in pred and 'poses3d' in pred:
            keypoints_2d = pred['poses2d'].numpy()
            keypoints_3d = pred['poses3d'].numpy()
            boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()

            if keypoints_2d.shape[0] == 0:
                print(f"No pose detected in frame {frame_count}")
                out_2d.write(frame)
                blank_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
                out_3d.write(blank_frame)
            else:
                if frame_count % frame_interval == 0:
                    visualize(frame_rgb_resized, boxes, keypoints_3d, keypoints_2d[:, :, :2], 
                             skeleton_edges, frame_count, plot_dir, display=False)

                for i in range(keypoints_2d.shape[0]):
                    if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:
                        kpt_2d = keypoints_2d[i]
                        kpt_3d = keypoints_3d[i]
                        kpt_2d[:, 0] *= original_width / 640
                        kpt_2d[:, 1] *= original_height / 480
                        frame = draw_skeleton(frame, kpt_2d, skeleton_edges)
                        person_label = f"Person {i + 1}"
                        label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))
                        cv2.putText(frame, person_label, label_position, 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
                        
                        for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):
                            data_list.append([
                                frame_count, i, j, 
                                kpt_2d[j][0], kpt_2d[j][1], 
                                kpt_3d[j][0] * 1000, kpt_3d[j][1] * 1000, kpt_3d[j][2] * 1000
                            ])
                
                out_2d.write(frame)
                
                temp_image_path = render_3d_skeleton(keypoints_3d, person_ids, skeleton_edges, 
                                                   temp_dir, frame_count, frame_width, frame_height)
                if temp_image_path:
                    plot_image = cv2.imread(temp_image_path)
                    if plot_image is not None:
                        plot_image = cv2.resize(plot_image, (frame_width, frame_height))
                        out_3d.write(plot_image)
                    else:
                        print(f"Failed to read 3D plot image for frame {frame_count}")
                        out_3d.write(np.zeros((frame_height, frame_width, 3), dtype=np.uint8))
                else:
                    out_3d.write(np.zeros((frame_height, frame_width, 3), dtype=np.uint8))
        else:
            out_2d.write(frame)
            out_3d.write(np.zeros((frame_height, frame_width, 3), dtype=np.uint8))

        frame_count += 1

    cap.release()
    out_2d.release()
    out_3d.release()
    shutil.rmtree(temp_dir)
    print(f"Processed 2D video saved as {output_2d_video}")
    print(f"Processed 3D video saved as {output_3d_video}")

    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])
    df.to_excel(output_excel, index=False)
    print(f"Pose data saved as {output_excel}")
    return df

def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }
    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None
    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])
    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])
        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')
        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))
    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}
    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (mm)': stride_length_left_2d,
        'Stride Length Right (mm)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (mm/s)': gait_speed_left_2d,
        'Gait Speed Right (mm/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }

def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }
    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None
    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])
    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])
        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')
        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))
    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}
    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (mm)': stride_length_left_3d,
        'Stride Length Right (mm)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (mm/s)': gait_speed_left_3d,
        'Gait Speed Right (mm/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }

def main():
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\Mediapipe\\walk.mp4"
    output_excel = "2dand3dposedatawithlabelschanged.xlsx"
    
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")
    
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()
    
    df = process_video(video_path, model, joint_edges, output_excel=output_excel)
    
    unique_persons = df['Person'].unique()
    pixel_to_mm_factors = []
    for person_id in unique_persons:
        df_person = df[df['Person'] == person_id]
        left_hip_2d = df_person[df_person['Joint'] == 1][['Frame', 'X_2D', 'Y_2D']]
        left_ankle_2d = df_person[df_person['Joint'] == 7][['Frame', 'X_2D', 'Y_2D']]
        left_hip_3d = df_person[df_person['Joint'] == 1][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]
        left_ankle_3d = df_person[df_person['Joint'] == 7][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]
        hip_ankle_2d = left_hip_2d.merge(left_ankle_2d, on='Frame', suffixes=('_hip', '_ankle'))
        hip_ankle_3d = left_hip_3d.merge(left_ankle_3d, on='Frame', suffixes=('_hip', '_ankle'))
        distances_2d = []
        distances_3d = []
        for idx in range(len(hip_ankle_2d)):
            hip_2d = hip_ankle_2d.iloc[idx][['X_2D_hip', 'Y_2D_hip']].values
            ankle_2d = hip_ankle_2d.iloc[idx][['X_2D_ankle', 'Y_2D_ankle']].values
            dist_2d = distance.euclidean(hip_2d, ankle_2d)
            hip_3d = hip_ankle_3d.iloc[idx][['X_3D_hip', 'Y_3D_hip', 'Z_3D_hip']].values
            ankle_3d = hip_ankle_3d.iloc[idx][['X_3D_ankle', 'Y_3D_ankle', 'Z_3D_ankle']].values
            dist_3d = distance.euclidean(hip_3d, ankle_3d)
            if dist_2d > 0:
                pixel_to_mm = dist_3d / dist_2d
                pixel_to_mm_factors.append(pixel_to_mm)
                distances_2d.append(dist_2d)
                distances_3d.append(dist_3d)
    pixel_to_mm = np.mean(pixel_to_mm_factors) if pixel_to_mm_factors else 900 / 500
    print(f"Computed pixel-to-mm conversion factor: {pixel_to_mm:.4f} mm/pixel")
    
    df['X_2D'] = df['X_2D'] * pixel_to_mm
    df['Y_2D'] = df['Y_2D'] * pixel_to_mm
    updated_excel = "2dand3dposedatawithlabelschanged_converted.xlsx"
    df.to_excel(updated_excel, index=False)
    print(f"Updated pose data saved as {updated_excel}")
    
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
    
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("Gait metrics saved to 'gait_metrics_per_person.xlsx'")

if __name__ == '__main__':
    main()

Model loaded successfully!
No pose detected in frame 25
No pose detected in frame 26
No pose detected in frame 27
No pose detected in frame 36
No pose detected in frame 37
No pose detected in frame 129
No pose detected in frame 130
No pose detected in frame 142
Processed 2D video saved as output_video3d_withlabelschanged.mp4
Processed 3D video saved as output_3d_pose.mp4
Pose data saved as 2dand3dposedatawithlabelschanged.xlsx
Computed pixel-to-mm conversion factor: 2001.6513 mm/pixel
Updated pose data saved as 2dand3dposedatawithlabelschanged_converted.xlsx
Gait metrics saved to 'gait_metrics_per_person.xlsx'
